<a href="https://colab.research.google.com/github/moodlep/rl-playground/blob/main/ppo/colab_notebooks/PPO_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [1]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]

     |████████████████████████████████| 448 kB 6.8 MB/s 


In [2]:
!pip install stable-baselines3

     |████████████████████████████████| 174 kB 8.9 MB/s 


In [3]:
import os
import Box2D
import pyglet
import imageio
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [4]:
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from stable_baselines3.common.vec_env import SubprocVecEnv
import multiprocessing


# Test Env

In [5]:
env_id = "LunarLanderContinuous-v2"
env = gym.make(env_id)


In [6]:
env.reset()

array([-0.00528822,  1.4155146 , -0.5356595 ,  0.20417766,  0.00613458,
        0.12133496,  0.        ,  0.        ], dtype=float32)

In [7]:
env.action_space.shape, env.observation_space.shape

((2,), (8,))

In [8]:
for episode in range(1): 
    observation = env.reset()
    for step in range(1):
        action = env.action_space.sample()  # or given a custom model, action = policy(observation)
        observation, reward, done, info = env.step(action)
        print(observation, reward, done, info, action)

[-0.01534519  1.4182618  -0.7760848   0.15024614  0.01759557  0.17398687
  0.          0.        ] -0.5834078874543138 False {} [-0.33294398 -0.08262137]


# ActorCritic Model

In [9]:
class ActorCritic(nn.Module):

  def __init__(self, state_dim, action_dim, std_init):
    super(ActorCritic,self).__init__()

    # TBD switch to variable std

    self.action_dim = action_dim
    self.state_dim = state_dim
    self.critic = nn.Sequential(
        nn.Linear(self.state_dim, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, 1)
    )

    self.actor = nn.Sequential(
        nn.Linear(self.state_dim, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, self.action_dim),
        nn.Tanh()
    )

    # covariance for Multivariate Normal policy
    self.action_vars = torch.full((self.action_dim,), std_init * std_init)
    self.cov_mat = torch.diag(self.action_vars).unsqueeze(dim=0)  # do we need the unsqueeze? 

  def get_action(self, state):
    means = self.actor(state)
    policy = torch.distributions.MultivariateNormal(means, self.cov_mat)

    return policy.sample()

    
  def get_value(self,state):
    return self.critic(state)

  



In [10]:
ac = ActorCritic(env.observation_space.shape[0], env.action_space.shape[0], 0.05)

In [11]:
states = torch.rand

In [12]:
pi = ac.get_action(torch.tensor([observation, observation]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


# Buffer

In [13]:
class Buffer:
  def __init__(self,state_dim, action_dim ):
    #self.num_actors = num_actors
    #self.max_timesteps = max_timestep
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.buffer = torch.zeros_like([])
  def add_transion(self):
    return None

# Envs

In [14]:
def make_env(env_id: str, rank: int, seed: int = 0):
  def _init():
    env = gym.make(env_id)
    env.seed(seed + rank)
    return env
  torch.manual_seed(seed)
  return _init

num_cpu = 4
env_p = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])


In [15]:
obs = torch.tensor(env_p.reset())
obs

tensor([[-5.9156e-04,  1.4135e+00, -5.9936e-02,  1.1277e-01,  6.9229e-04,
          1.3576e-02,  0.0000e+00,  0.0000e+00],
        [-5.2567e-03,  1.3989e+00, -5.3248e-01, -5.3348e-01,  6.0981e-03,
          1.2061e-01,  0.0000e+00,  0.0000e+00],
        [-4.0088e-03,  1.4072e+00, -4.0605e-01, -1.6675e-01,  4.6519e-03,
          9.1977e-02,  0.0000e+00,  0.0000e+00],
        [-7.2308e-03,  1.4090e+00, -7.3242e-01, -8.7516e-02,  8.3855e-03,
          1.6590e-01,  0.0000e+00,  0.0000e+00]])

In [16]:
actions = ac.get_action(obs).numpy()
actions

array([[-0.02091479,  0.021314  ],
       [-0.21645916,  0.02415736],
       [-0.16103895, -0.06214209],
       [-0.09780074,  0.02694974]], dtype=float32)

In [17]:
env_p.step_async(actions)

In [18]:
env_p.step_wait()

(array([[-1.18331914e-03,  1.41541684e+00, -5.98544367e-02,
          8.70871544e-02,  1.36317231e-03,  1.34195890e-02,
          0.00000000e+00,  0.00000000e+00],
        [-1.05140684e-02,  1.38633800e+00, -5.31758964e-01,
         -5.59106767e-01,  1.20580401e-02,  1.19209744e-01,
          0.00000000e+00,  0.00000000e+00],
        [-8.01782589e-03,  1.40283966e+00, -4.05504376e-01,
         -1.92408934e-01,  9.19678714e-03,  9.09073725e-02,
          0.00000000e+00,  0.00000000e+00],
        [-1.44620892e-02,  1.40640640e+00, -7.31432617e-01,
         -1.13186173e-01,  1.65834818e-02,  1.63975552e-01,
          0.00000000e+00,  0.00000000e+00]], dtype=float32),
 array([ 1.94054617, -1.12638708, -1.01123009, -0.82140293]),
 array([False, False, False, False]),
 ({}, {}, {}, {}))

In [19]:
env_p.step_async(actions)
observation, reward, done, information = env_p.step_wait()
print(len(done))



4


In [20]:
env_p.action_space.shape

(2,)

# PPO Class: 

* sort out seeds
* to.device()


In [21]:
# Actual cores in Colab is too low so we will stick with our values!
num_cpus = multiprocessing.cpu_count()
num_cpus

2

In [22]:
config = {
    'std_init': 0.05,
    'env_id': 'LunarLanderContinuous-v2',
    'num_workers': 4,  # rank (seed) / envs / N
    'num_epochs': 10, # K number of 
    'num_iterations': 10, # number of times we collect a dataset 
    'max_timesteps': 500, # T
    'epsilon': 0.2,  # clipping radius

}

In [26]:
torch.zeros((4,500, obs.shape[0])).shape


torch.Size([4, 500, 4])

In [34]:
class PPO:
  def __init__(self, config):

    self.config = config
    # self.num_cpus = multiprocessing.cpu_count()

    self.envs = SubprocVecEnv([make_env(config['env_id'], i) for i in range(self.config['num_workers'])])

    self.action_dim = self.envs.action_space.shape[0]
    self.state_dim = self.envs.observation_space.shape[0]

    self.model = ActorCritic(self.state_dim, self.action_dim, config['std_init'])

  def create_rollout(self):

    pass

  def train(self): 

    # num_iterations = Number of updates: 
    for it in range(self.config['num_iterations']):

      obs_batch = torch.zeros((self.config['max_timesteps'], self.config['num_workers'],  self.envs.observation_space.shape[0]))
      action_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], self.envs.action_space.shape[0]))
      reward_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], 1))
      done_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], 1))
      next_obs_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], self.envs.observation_space.shape[0]))
      ratio_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], 1))
      advantage_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], 1))
      returns_batch = torch.zeros((self.config['max_timesteps'],self.config['num_workers'], 1))

      obs = self.envs.reset()

      # capture NT rollouts 
      for t in range(self.config['max_timesteps']):
        actions = self.model.get_action(torch.tensor(obs)).numpy()
        self.envs.step_async(actions)
        next_obs, rewards, dones, infos = self.envs.step_wait()
        
        obs_batch[t] = torch.tensor(obs)
        action_batch[t] = torch.tensor(actions)
        reward_batch[t] = torch.tensor(rewards)
        done_batch[t] = torch.tensor(dones)
        next_obs_batch[t] = torch.tensor(next_obs)

        obs = next_obs

      # Calculate returns to go: STOPPED HERE!!
      for t in range(self.config['max_timesteps']-1, -1, -1):
        for worker in range(self.config['num_workers']):
          pass


      # Calculate Advantage: 





In [33]:
a = np.array([1,2,3,4])

In [31]:
tmp = torch.zeros([1])
tmp[0]=True
tmp

tensor([1.])

In [32]:
for t in range(10-1, -1, -1):
  print(t)

9
8
7
6
5
4
3
2
1
0


## covariance stuff

Src: https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/PPO_colab.ipynb

In [ ]:
torch.full((2,), 3 * 3)

In [ ]:
torch.diag(torch.full((2,), 3 * 3)).unsqueeze(dim=0)